In [9]:
import math

def log_safe(val):
    return -float('inf') if val == 0 else math.log(val)

def compute_log_prob(traj, obs, pi, a, b):
    if len(traj) != len(obs):
        raise ValueError("Trajectory and observation must match in length")
    
    score = 0.0
    for t in range(len(obs)):
        state = traj[t]
        sym = obs[t]
        
        if t == 0:
            score += log_safe(pi[state])
        else:
            prev = traj[t - 1]
            score += log_safe(a[prev].get(state, 0))
        
        score += log_safe(b[state].get(sym, 0))
    
    if traj[-1] == 'I' and 'end' in a['I']:
        score += log_safe(a['I']['end'])
    
    return score

states = ['E', '5', 'I']

initial = {
    'E': 1.0,
    '5': 0.0,
    'I': 0.0
}

transitions = {
    'E': {'E': 0.9, '5': 0.1},
    '5': {'I': 1.0},
    'I': {'I': 0.9, 'end': 0.1}
}

emissions = {
    'E': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
    '5': {'A': 0.05, 'C': 0.0, 'G': 0.95, 'T': 0.0},
    'I': {'A': 0.4, 'C': 0.1, 'G': 0.1, 'T': 0.4}
}

trajectory = "EEEEEEEEEEEEEEEEEE5IIIIIII"
observed = "CTTCATGTGAAAGCAGACGTAAGTCA"

score = compute_log_prob(trajectory, observed, initial, transitions, emissions)
print(f"Log probability of the given path: {score:.2f}")

Log probability of the given path: -41.22


In [10]:
def viterbi(obs, states, pi, a, b):
    T = len(obs)
    V = [{} for _ in range(T)]
    B = [{} for _ in range(T)]

    sym = obs[0]
    for s in states:
        e = b[s].get(sym, 0)
        V[0][s] = log_safe(pi[s]) + log_safe(e)
        B[0][s] = None

    for t in range(1, T):
        sym = obs[t]
        for curr in states:
            e = b[curr].get(sym, 0)
            if e == 0:
                continue

            best = -float('inf')
            best_prev = None

            for prev in states:
                trans = a.get(prev, {}).get(curr, 0)
                if trans > 0:
                    score = V[t - 1].get(prev, -float('inf')) + log_safe(trans) + log_safe(e)
                    if score > best:
                        best = score
                        best_prev = prev

            if best_prev is not None:
                V[t][curr] = best
                B[t][curr] = best_prev

    last = T - 1
    final = max(V[last], key=V[last].get)
    path = [final]
    curr = final

    for t in range(last, 0, -1):
        prev = B[t].get(curr)
        if prev is None:
            break
        path.insert(0, prev)
        curr = prev

    return ''.join(path)

result = viterbi(
    observed,
    states,
    initial,
    transitions,
    emissions
)

print(f"Most likely path: {result}")

Most likely path: EEEEEEEEEEEEEEEEEEEEEEEEEE
